# Feature Engineering

Only the historical AQI and air pollutant data has been stored in Hopsworks. This is because time series forecasting involves generating features from historical data that are then used to train the model and predict new values from. In other words, we need the historical data at predict time to generate the features on which we predict the future AQI. One thing we need to be extra careful of is leakage. We need to be careful that we only train and predict on features that are known at the time of prediction. In this notebook, I go through the process of engineering features that we will use to train an XGBoost classifier on. The algorithms will then be combined into a function that I can use in a separate notebook where I actually create the features and train a model.

### Import Data

We import data from Hopsworks. I've created a training set that is actually just the entire dataset, and then I download it so that I can use it here in the notebook.

In [1]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/14486
Connected. Call `.close()` to terminate connection gracefully.


In [2]:
# Load feature group.
zip_code = '60603'  # Chicago
country_code = 'US'
city = 'Chicago'

fg_name = f'aqi_{city}_{zip_code}'.lower()

aqi_online_fg = fs.get_feature_group(fg_name, version=1)

not_features = ['date', 'lat', 'lon']

ds_query = aqi_online_fg.select_except(not_features)

In [3]:
ds_query.show(5, online=True)

,co,no,no2,o3,so2,pm2_5,pm10,nh3,datetime,aqi,id
0,347.14,10.39,21.94,39.70,8.46,7.21,11.43,2.69,2020-11-28 12:00:00,1,36
1,440.60,16.99,26.39,26.82,10.97,18.80,24.32,4.37,2020-12-03 12:00:00,2,156
2,340.46,5.03,25.36,42.20,10.97,16.63,19.86,2.50,2020-12-03 16:00:00,2,160
3,273.71,0.02,14.91,41.84,5.42,11.97,13.51,0.63,2020-12-04 06:00:00,2,174
4,243.66,1.02,11.31,80.11,4.71,1.38,2.84,0.76,2020-12-05 12:00:00,2,204


In [4]:
fv_name = f'{fg_name}_fv'

feature_view = fs.get_feature_view(name=fv_name, version=1)

In [5]:
import datetime
import pandas as pd

end_date = pd.to_datetime(fs.sql(f"SELECT MAX(`datetime`) FROM `{fg_name}_1`", online=True).values[0][0])
start_date = pd.to_datetime(fs.sql(f"SELECT MIN(`datetime`) FROM `{fg_name}_1`", online=True).values[0][0])

start_date_str = start_date.strftime('%Y-%m-%d %H:%M:%S')
end_date_str = end_date.strftime('%Y-%m-%d %H:%M:%S')

print(start_date_str, end_date_str)

2020-11-27 00:00:00 2023-01-19 14:00:00


In [6]:
train_x, train_y = feature_view.get_training_data(1)

# need to convert datetime from strings
train_x.datetime = pd.to_datetime(train_x.datetime)

# data points are not in order
train_x = train_x.sort_values("datetime")
train_y = train_y.reindex(train_x.index)

# need to remove time zone information
train_x['datetime'] = train_x['datetime'].dt.tz_localize(None)

# use the datetime as index now
train_x = train_x.reset_index(drop=True)
train_x = train_x.set_index('datetime')

train_y = train_y.reset_index(drop=True)
train_y = train_y.set_index(train_x.index)
train_y['aqi'] = train_y['aqi']-1  # xgboost requires zero indexed categories for classification

In [7]:
# concat
df = pd.concat([train_x, train_y], axis=1)
df = df.drop(columns=['id'])

In [8]:
df.head()

,co,no,no2,o3,so2,pm2_5,pm10,nh3,aqi
datetime,,,,,,,,,
2020-11-27 12:00:00,534.06,26.60,45.93,3.84,15.97,19.48,28.15,2.72,1
2020-11-27 13:00:00,527.38,27.49,45.93,3.44,16.93,19.42,27.72,2.76,1
2020-11-27 14:00:00,433.92,12.63,39.76,15.20,11.80,17.89,23.37,1.81,1
2020-11-27 15:00:00,380.52,7.82,29.47,30.76,8.70,16.56,20.47,1.60,1
2020-11-27 16:00:00,353.81,5.31,26.73,37.55,7.87,14.31,17.50,1.69,1


### Features from the Timestamp

Alright so the data is imported and now we can get started making some features. The first ones I want to make are just from the time stamp:

In [9]:
df['hour'] = df.index.hour
df['dayofweek'] = df.index.dayofweek
df['quarter'] = df.index.quarter
df['month'] = df.index.month
df['year'] = df.index.year
df['dayofyear'] = df.index.dayofyear
df['dayofmonth'] = df.index.day
df['weekofyear'] = df.index.isocalendar().week.astype('int')

In [10]:
df.head()

,co,no,no2,o3,so2,pm2_5,pm10,nh3,aqi,hour,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear
datetime,,,,,,,,,,,,,,,,,
2020-11-27 12:00:00,534.06,26.60,45.93,3.84,15.97,19.48,28.15,2.72,1,12,4,4,11,2020,332,27,48
2020-11-27 13:00:00,527.38,27.49,45.93,3.44,16.93,19.42,27.72,2.76,1,13,4,4,11,2020,332,27,48
2020-11-27 14:00:00,433.92,12.63,39.76,15.20,11.80,17.89,23.37,1.81,1,14,4,4,11,2020,332,27,48
2020-11-27 15:00:00,380.52,7.82,29.47,30.76,8.70,16.56,20.47,1.60,1,15,4,4,11,2020,332,27,48
2020-11-27 16:00:00,353.81,5.31,26.73,37.55,7.87,14.31,17.50,1.69,1,16,4,4,11,2020,332,27,48


### Lagging Features

Lag features are information about a previous time step of the time series. We use them because the the past values of a variable are likely to be predictive of future variables. Past values of other predictive features can also be useful for our forecast. Thus, in forecasting, it is common practice to create lag features from time series data and use them as input to machine learning algorithms.

In this case, we can lag a lot of things to create features like the various pollutant concentrations and the AQI. I know that my end goal is to predict the AQI for 3 days into the future. This means that the minimum lag I can do is 3 days. Later, I'll create some features with window functions where it'll be important to consider the 3-day shift in order to avoid a data leak.

In [11]:
df.columns

Index(['co', 'no', 'no2', 'o3', 'so2', 'pm2_5', 'pm10', 'nh3', 'aqi', 'hour',
       'dayofweek', 'quarter', 'month', 'year', 'dayofyear', 'dayofmonth',
       'weekofyear'],
      dtype='object')

We can use the `shift()` function to lag the features by a given amount.

In [12]:
features_to_lag = ['co', 'no', 'no2', 'o3', 'so2', 'pm2_5', 'pm10', 'nh3', 'aqi']

for feature in features_to_lag:
    # lag feature by 3 days
    new_feature_name = feature + '_lag3d'
    df[new_feature_name] = df[feature].shift(freq='3D', axis=0)
    
    # lag feature by 5 days
    new_feature_name = feature + '_lag5d'
    df[new_feature_name] = df[feature].shift(freq='5D', axis=0)

    # lag feature by 9 days
    new_feature_name = feature + '_lag9d'
    df[new_feature_name] = df[feature].shift(freq='9D', axis=0)

In [13]:
df.head()

,co,no,no2,o3,so2,pm2_5,pm10,nh3,aqi,hour,...,pm2_5_lag9d,pm10_lag3d,pm10_lag5d,pm10_lag9d,nh3_lag3d,nh3_lag5d,nh3_lag9d,aqi_lag3d,aqi_lag5d,aqi_lag9d
datetime,,,,,,,,,,,,,,,,,,,,,
2020-11-27 12:00:00,534.06,26.60,45.93,3.84,15.97,19.48,28.15,2.72,1,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-27 13:00:00,527.38,27.49,45.93,3.44,16.93,19.42,27.72,2.76,1,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-27 14:00:00,433.92,12.63,39.76,15.20,11.80,17.89,23.37,1.81,1,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-27 15:00:00,380.52,7.82,29.47,30.76,8.70,16.56,20.47,1.60,1,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-27 16:00:00,353.81,5.31,26.73,37.55,7.87,14.31,17.50,1.69,1,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df.columns

Index(['co', 'no', 'no2', 'o3', 'so2', 'pm2_5', 'pm10', 'nh3', 'aqi', 'hour',
       'dayofweek', 'quarter', 'month', 'year', 'dayofyear', 'dayofmonth',
       'weekofyear', 'co_lag3d', 'co_lag5d', 'co_lag9d', 'no_lag3d',
       'no_lag5d', 'no_lag9d', 'no2_lag3d', 'no2_lag5d', 'no2_lag9d',
       'o3_lag3d', 'o3_lag5d', 'o3_lag9d', 'so2_lag3d', 'so2_lag5d',
       'so2_lag9d', 'pm2_5_lag3d', 'pm2_5_lag5d', 'pm2_5_lag9d', 'pm10_lag3d',
       'pm10_lag5d', 'pm10_lag9d', 'nh3_lag3d', 'nh3_lag5d', 'nh3_lag9d',
       'aqi_lag3d', 'aqi_lag5d', 'aqi_lag9d'],
      dtype='object')

Notice that there are a lot of 'NaN's at the beginning of the dataframe now. That is because we don't have previous data from which to generate a lag feature. XGBoost will handle the missing values automatically. We can look at the tail of the data and see that we have data:

In [15]:
df.tail()

,co,no,no2,o3,so2,pm2_5,pm10,nh3,aqi,hour,...,pm2_5_lag9d,pm10_lag3d,pm10_lag5d,pm10_lag9d,nh3_lag3d,nh3_lag5d,nh3_lag9d,aqi_lag3d,aqi_lag5d,aqi_lag9d
datetime,,,,,,,,,,,,,,,,,,,,,
2023-01-18 22:00:00,260.35,0.03,11.65,44.70,3.01,1.82,2.43,0.36,0,22,...,18.97,11.60,6.81,24.28,1.81,1.84,1.19,0.0,0.0,1.0
2023-01-18 23:00:00,280.38,0.06,14.05,34.33,2.89,2.61,3.21,0.38,0,23,...,18.69,10.36,6.39,23.12,1.66,1.65,0.78,0.0,0.0,1.0
2023-01-19 00:00:00,293.73,0.06,12.68,33.62,1.65,2.64,3.13,0.34,0,0,...,17.77,8.82,6.35,21.47,1.49,1.60,0.63,1.0,0.0,1.0
2023-01-19 01:00:00,290.39,0.03,9.85,36.84,1.55,1.94,2.25,0.25,0,1,...,15.75,6.96,5.72,18.51,1.22,1.46,0.45,1.0,0.0,1.0
2023-01-19 02:00:00,290.39,0.02,9.60,40.05,1.88,1.96,2.27,0.25,0,2,...,14.02,6.15,5.49,15.84,1.14,1.42,0.39,1.0,0.0,1.0


### Rolling Window Features

Window features are the result of window operations over the variables. Here I calculate the rolling maximum, mean, and standard deviation over a window of 12 hours, and then lag the results by 3 days. I think the standard deviation is particularly important because it will help capture the volatility in the AQI data.

In [16]:
window = 12  # hours
df['aqi_max_lag_3d'] = df['aqi'].rolling(window=window).agg(['max']).shift(freq='3D', axis=0)
df['aqi_mean_lag_3d'] = df['aqi'].rolling(window=window).agg(['mean']).shift(freq='3D', axis=0)
df['aqi_std_lag_3d'] = df['aqi'].rolling(window=window).agg(['std']).shift(freq='3D', axis=0)

In [17]:
df.tail()

,co,no,no2,o3,so2,pm2_5,pm10,nh3,aqi,hour,...,pm10_lag9d,nh3_lag3d,nh3_lag5d,nh3_lag9d,aqi_lag3d,aqi_lag5d,aqi_lag9d,aqi_max_lag_3d,aqi_mean_lag_3d,aqi_std_lag_3d
datetime,,,,,,,,,,,,,,,,,,,,,
2023-01-18 22:00:00,260.35,0.03,11.65,44.70,3.01,1.82,2.43,0.36,0,22,...,24.28,1.81,1.84,1.19,0.0,0.0,1.0,1.0,0.666667,0.492366
2023-01-18 23:00:00,280.38,0.06,14.05,34.33,2.89,2.61,3.21,0.38,0,23,...,23.12,1.66,1.65,0.78,0.0,0.0,1.0,1.0,0.583333,0.514929
2023-01-19 00:00:00,293.73,0.06,12.68,33.62,1.65,2.64,3.13,0.34,0,0,...,21.47,1.49,1.60,0.63,1.0,0.0,1.0,1.0,0.583333,0.514929
2023-01-19 01:00:00,290.39,0.03,9.85,36.84,1.55,1.94,2.25,0.25,0,1,...,18.51,1.22,1.46,0.45,1.0,0.0,1.0,1.0,0.583333,0.514929
2023-01-19 02:00:00,290.39,0.02,9.60,40.05,1.88,1.96,2.27,0.25,0,2,...,15.84,1.14,1.42,0.39,1.0,0.0,1.0,1.0,0.583333,0.514929


Now that I have all the features I want, I can drop the historical data, since those are the features that I will not be able to use in prediction:

In [18]:
df = df.drop(columns=['co', 'no', 'no2', 'o3', 'so2', 'pm2_5', 'pm10', 'nh3'])
df.tail()

,aqi,hour,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,co_lag3d,...,pm10_lag9d,nh3_lag3d,nh3_lag5d,nh3_lag9d,aqi_lag3d,aqi_lag5d,aqi_lag9d,aqi_max_lag_3d,aqi_mean_lag_3d,aqi_std_lag_3d
datetime,,,,,,,,,,,,,,,,,,,,,
2023-01-18 22:00:00,0,22,2,1,1,2023,18,18,3,317.10,...,24.28,1.81,1.84,1.19,0.0,0.0,1.0,1.0,0.666667,0.492366
2023-01-18 23:00:00,0,23,2,1,1,2023,18,18,3,300.41,...,23.12,1.66,1.65,0.78,0.0,0.0,1.0,1.0,0.583333,0.514929
2023-01-19 00:00:00,0,0,3,1,1,2023,19,19,3,287.06,...,21.47,1.49,1.60,0.63,1.0,0.0,1.0,1.0,0.583333,0.514929
2023-01-19 01:00:00,0,1,3,1,1,2023,19,19,3,267.03,...,18.51,1.22,1.46,0.45,1.0,0.0,1.0,1.0,0.583333,0.514929
2023-01-19 02:00:00,0,2,3,1,1,2023,19,19,3,260.35,...,15.84,1.14,1.42,0.39,1.0,0.0,1.0,1.0,0.583333,0.514929


Now let's combine everything into a single function:

In [19]:
def createFeatures(data: pd.DataFrame) -> pd.DataFrame:
    df = data.copy()
    # add date features
    df['hour'] = df.index.hour
    df['dayofweek'] = df.index.dayofweek
    df['quarter'] = df.index.quarter
    df['month'] = df.index.month
    df['year'] = df.index.year
    df['dayofyear'] = df.index.dayofyear
    df['dayofmonth'] = df.index.day
    df['weekofyear'] = df.index.isocalendar().week.astype('int')
    
    # add lag features
    features_to_lag = ['co', 'no', 'no2', 'o3', 'so2', 'pm2_5', 'pm10', 'nh3', 'aqi']

    for feature in features_to_lag:
        # lag feature by 3 days
        new_feature_name = feature + '_lag3d'
        df[new_feature_name] = df[feature].shift(freq='3D', axis=0)

        # lag feature by 5 days
        new_feature_name = feature + '_lag5d'
        df[new_feature_name] = df[feature].shift(freq='5D', axis=0)

        # lag feature by 9 days
        new_feature_name = feature + '_lag9d'
        df[new_feature_name] = df[feature].shift(freq='9D', axis=0)
        
        
    window = 12  # hours
    df['aqi_max_lag_3d'] = df['aqi'].rolling(window=window).agg(['max']).shift(freq='3D', axis=0)
    df['aqi_mean_lag_3d'] = df['aqi'].rolling(window=window).agg(['mean']).shift(freq='3D', axis=0)
    df['aqi_std_lag_3d'] = df['aqi'].rolling(window=window).agg(['std']).shift(freq='3D', axis=0)
    
    # drop the historical features
    df = df.drop(columns=['co', 'no', 'no2', 'o3', 'so2', 'pm2_5', 'pm10', 'nh3'])
    
    return df

In [20]:
# test
# concat
df = pd.concat([train_x, train_y], axis=1)
df = df.drop(columns=['id'])

In [21]:
df

,co,no,no2,o3,so2,pm2_5,pm10,nh3,aqi
datetime,,,,,,,,,
2020-11-27 12:00:00,534.06,26.60,45.93,3.84,15.97,19.48,28.15,2.72,1
2020-11-27 13:00:00,527.38,27.49,45.93,3.44,16.93,19.42,27.72,2.76,1
2020-11-27 14:00:00,433.92,12.63,39.76,15.20,11.80,17.89,23.37,1.81,1
2020-11-27 15:00:00,380.52,7.82,29.47,30.76,8.70,16.56,20.47,1.60,1
2020-11-27 16:00:00,353.81,5.31,26.73,37.55,7.87,14.31,17.50,1.69,1
...,...,...,...,...,...,...,...,...,...
2023-01-18 22:00:00,260.35,0.03,11.65,44.70,3.01,1.82,2.43,0.36,0
2023-01-18 23:00:00,280.38,0.06,14.05,34.33,2.89,2.61,3.21,0.38,0
2023-01-19 00:00:00,293.73,0.06,12.68,33.62,1.65,2.64,3.13,0.34,0


In [22]:
df = createFeatures(df)
df.tail()

,aqi,hour,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,co_lag3d,...,pm10_lag9d,nh3_lag3d,nh3_lag5d,nh3_lag9d,aqi_lag3d,aqi_lag5d,aqi_lag9d,aqi_max_lag_3d,aqi_mean_lag_3d,aqi_std_lag_3d
datetime,,,,,,,,,,,,,,,,,,,,,
2023-01-18 22:00:00,0,22,2,1,1,2023,18,18,3,317.10,...,24.28,1.81,1.84,1.19,0.0,0.0,1.0,1.0,0.666667,0.492366
2023-01-18 23:00:00,0,23,2,1,1,2023,18,18,3,300.41,...,23.12,1.66,1.65,0.78,0.0,0.0,1.0,1.0,0.583333,0.514929
2023-01-19 00:00:00,0,0,3,1,1,2023,19,19,3,287.06,...,21.47,1.49,1.60,0.63,1.0,0.0,1.0,1.0,0.583333,0.514929
2023-01-19 01:00:00,0,1,3,1,1,2023,19,19,3,267.03,...,18.51,1.22,1.46,0.45,1.0,0.0,1.0,1.0,0.583333,0.514929
2023-01-19 02:00:00,0,2,3,1,1,2023,19,19,3,260.35,...,15.84,1.14,1.42,0.39,1.0,0.0,1.0,1.0,0.583333,0.514929


Hooray!